In [1]:
import pandas as pd
from auxiliary.filter_functions import filter_chandra, filter_archival, filter_gaia, filter_ned, filter_simbad, filter_erosita, filter_vizier
from astroquery.vizier import Vizier
from astroquery.simbad import Simbad, SimbadClass

In [9]:
detections = pd.read_csv('output/detections_w20.txt',
                         header=0, sep=' ', dtype=str)

detection = detections[detections['ObsId'] == '8490'].iloc[0]

filter_simbad(detection, True)

            MAIN_ID             RA            DEC    OTYPE    OTYPES
0  UCAC4 105-012588  05 29 02.8601  -69 04 29.953  EmLine*  Em*|PN|*


True

In [26]:
filterd = pd.read_csv('output/filtered_w20.csv', header=0, dtype=str)
obsids_after1 = pd.read_csv('obsid_lists/obsids_b+10_220401+.csv', header=0, dtype=str, sep=',', usecols=[1])
obsids_after2 =  pd.read_csv('obsid_lists/obsids_b-10_220401+.csv', header=0, dtype=str, sep=',', usecols=[1])
obsids_after = pd.concat([obsids_after1, obsids_after2], ignore_index=True)
print(f'after total: {len(obsids_after)}')

obsids_before1 = pd.read_csv('obsid_lists/obsids_b+10_220401-.csv', header=0, dtype=str, sep=',', usecols=[1])	
obsids_before2 = pd.read_csv('obsid_lists/obsids_b-10_220401-.csv', header=0, dtype=str, sep=',', usecols=[1])
obsids_before = pd.concat([obsids_before1, obsids_before2], ignore_index=True)
print(f'before total: {len(obsids_before)}')

filtered_post = filterd[filterd['ObsId'].isin(obsids_after['Obs ID'])]
filtered_pre = filterd[filterd['ObsId'].isin(obsids_before['Obs ID'])]
filtered_post.to_csv('output/filtered_w20_new.csv', index=False)
filtered_pre.to_csv('output/filtered_w20_old.csv', index=False)

print(f'after detec: {len(filtered_post)}')
print(f'before detec: {len(filtered_pre)}')

filtered_post_all_no = filtered_post[
    (filtered_post['gaia_match'] == 'no') &
    (filtered_post['simbad_match'] == 'no') &
    (filtered_post['ned_match'] == 'no') &
    (filtered_post['archival_match'] == 'no') &
    (filtered_post['chandra_match'] == 'no') &
    (filtered_post['erosita_match'] == 'no')
]

filtered_pre_all_no = filtered_pre[
    (filtered_pre['gaia_match'] == 'no') &
    (filtered_pre['simbad_match'] == 'no') &
    (filtered_pre['ned_match'] == 'no') &
    (filtered_pre['archival_match'] == 'no') &
    (filtered_pre['chandra_match'] == 'no') &
    (filtered_pre['erosita_match'] == 'no')
]

print(f'after no match: {len(filtered_post_all_no)}')
print(f'before no match: {len(filtered_pre_all_no)}')

after total: 1495
before total: 8304
after detec: 101
before detec: 1721
after no match: 30
before no match: 332
